# cherrypy

upload file in cherrypy

In [1]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
def readf(filename):
    
    # read file
    file=open(filename)
    read=file.read()
    file.seek(0)
   
    # lines in file
    line=1
    for word in read:
        if word=='\n':
            line+=1   

    # create array
    array=[]
    for i in range(line):
        array.append(file.readline())
    
    # remove punctuation
    punc = '''!()-[]{};:'"\, <>./?@#$%^&*_~'''
    for ele in read:  
        if ele in punc:  
            read = read.replace(ele, " ")  
 
    read=read.lower()
    
    # remove stopwords
    text_tokens = word_tokenize(read)
    tokens_without_sw = [word for word in text_tokens if not word in stopwords.words()]
    
    # remove digits
    tokens=[]
    for i in range(len(tokens_without_sw)):
                   if tokens_without_sw[i].isdigit()==False:
                       tokens.append(tokens_without_sw[i])
                    
    # stemming
    ps=PorterStemmer()
    for i in range(len(tokens)):
        tokens[i]=ps.stem(tokens[i])
        
    #set
    newset=set(tokens)
    
    #dictionary
    dt={}

    for i in range(line):
        check=array[i].lower()
        for item in newset:

            if item in check:
                if item not in dt:
                    dt[item]=[]

                if item in dt:
                    dt[item].append(i+1)

    return dt



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rameshwaram\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def setor(a,b):
    myset1=set(a)
    myset2=set(b)
    myset3= myset1|myset2
    return myset3

def setand(a,b):
    myset1=set(a)
    myset2=set(b)
    myset3= myset1&myset2
    return myset3

def precedence(op): 
      
    if op == '&' or op == '|': 
        return 2
    if op == '!': 
        return 1
    return 0

def applyOp(a,b,op):
    if op=='&':
        return setand(a,b)
    if op=='|':
        return setor(a,b)
    if op=='!':
        return setnot(a,b)
    
def setnot(a,b):
    myset1=set(a)
    myset2=set(b)
    myset3= myset1-myset2
    return myset3
    
    
operator=['|', '&','(',')','!']
def evaluate(tokens,filename,dt): 
    
    dt=readf(filename)
      
    values = []     
    ops = [] 
    i = 0
      
    while i < len(tokens):           
       
        if tokens[i] == ' ': 
            i += 1
            continue          
      
        elif tokens[i] == '(': 
            ops.append(tokens[i]) 
            
        elif tokens[i] not in operator : 
                      
            values.append(dt[tokens[i]]) 
        elif tokens[i] == ')': 
          
            while len(ops) != 0 and ops[-1] != '(': 
              
                val2 = values.pop() 
                val1 = values.pop() 
                op = ops.pop() 
                  
                values.append(applyOp(val1, val2, op)) 
              
            
            ops.pop() 
            
            
        else: 
          
           
            while (len(ops) != 0 and
                precedence(ops[-1]) >= precedence(tokens[i])): 
                          
                val2 = values.pop() 
                val1 = values.pop() 
                op = ops.pop() 
                  
                values.append(applyOp(val1, val2, op)) 
              
           
            ops.append(tokens[i]) 
          
        i += 1
    while len(ops) != 0: 
          
        val2 = values.pop() 
        val1 = values.pop() 
        op = ops.pop() 
                  
        values.append(applyOp(val1, val2, op)) 
      
    
    return values[0]     


In [9]:
import random
import string
import cherrypy
import nltk
import time
from cherrypy import tools


class Root(object):
   
    @cherrypy.expose
    def index(self):
        return """<html>
<head>
  <style>
    .container{
      height: 300px;
      width: 600px;
      background-color: maroon;
      margin-top: 12%;
      margin-left: 25%;
      color: white;
    }
    .container h2{
      font-size: 40px;
      text-align: center;  
      
    }
    .off{
      font-size: 25px;
      margin-left: 23%;
      color: yellow;
      background-color: ;
    }
    .on{
      font-size: 25px;
      border-top-right-radius: 25px;
      border-top-left-radius: 25px;
      border-bottom-left-radius: 25px;
      border-bottom-right-radius: 25px;
      margin-top: 20px;   }
  </style>
  </head>
<body>

  <div class="container">  
    <h2><u><i>Upload a file</i></u></h2>
    <form action="store" id="form" method="GET">
    <input class="off" type="file" name="myFile" /><br />
    <input style="margin-left: 50px; height: 40px; width: 500px;" class="on" type="text" name="search" placeholder="Enter the query............................................."><br>
    <input style="margin-left: 250px;" class="on" id=" submit" type="submit"/></div>
  </div>    
    </form>
  </div>
            
</body>
</html>"""
   
    
    @cherrypy.expose
    def store(self, myFile,search):
        c=[]
        
        tokens=word_tokenize(search)
        numb1=str(evaluate(tokens,myFile,c))
 
        out= """<html>
        <body>
        <h3 style="color:brown; font-size: 30px;">Output of query: %s</h3><br><br>
        
        <a id="shutdown"; href="./shutdown">Shutdown Server</a>
        </body>
        </html>
        
        """
        c=[]
        
        tokens=word_tokenize(search)
        numb1=str(evaluate(tokens,myFile,c))
        return out % (numb1)
  
    
    @cherrypy.expose

    def shutdown(self):
        cherrypy.engine.exit()

if __name__=="__main__":
    cherrypy.config.update({'server.socket_port': 8089})
    
    cherrypy.quickstart(Root())


[21/Jun/2020:22:13:30] ENGINE Listening for SIGTERM.
[21/Jun/2020:22:13:30] ENGINE Bus STARTING
CherryPy Checker:
The Application mounted at '' has an empty config.

[21/Jun/2020:22:13:30] ENGINE Set handler for console events.
[21/Jun/2020:22:13:30] ENGINE Started monitor thread 'Autoreloader'.
[21/Jun/2020:22:13:30] ENGINE Serving on http://127.0.0.1:8089
[21/Jun/2020:22:13:30] ENGINE Bus STARTED


127.0.0.1 - - [21/Jun/2020:22:13:32] "GET / HTTP/1.1" 200 1126 "http://localhost:8888/notebooks/cherrypy.ipynb" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.106 Safari/537.36"
127.0.0.1 - - [21/Jun/2020:22:13:48] "GET /store?myFile=romy.txt&search=%28first+%7C+second%29 HTTP/1.1" 200 234 "http://127.0.0.1:8089/" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.106 Safari/537.36"


[21/Jun/2020:22:13:51] ENGINE Bus STOPPING
[21/Jun/2020:22:13:52] ENGINE HTTP Server cherrypy._cpwsgi_server.CPWSGIServer(('127.0.0.1', 8089)) shut down
[21/Jun/2020:22:13:52] ENGINE Removed handler for console events.
[21/Jun/2020:22:13:52] ENGINE Stopped thread 'Autoreloader'.
[21/Jun/2020:22:13:52] ENGINE Bus STOPPED
[21/Jun/2020:22:13:52] ENGINE Bus EXITING
[21/Jun/2020:22:13:52] ENGINE Waiting for child threads to terminate...
[21/Jun/2020:22:13:52] ENGINE Bus EXITED


127.0.0.1 - - [21/Jun/2020:22:13:52] "GET /shutdown HTTP/1.1" 200 - "http://127.0.0.1:8089/store?myFile=romy.txt&search=%28first+%7C+second%29" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.106 Safari/537.36"


[21/Jun/2020:22:13:52] ENGINE Waiting for thread CP Server Thread-85.
